<a href="https://colab.research.google.com/github/riasonalker1906/aging-farmer-demographic-productivity/blob/main/Choropleth_map_of_average_age_by_county.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --quiet gcsfs google-cloud-storage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 7.0 MB/s eta 0:00:00


# Loading Data into Environment

In [ ]:
%%capture
## Authenticate your Google account to access Google Cloud services. This will allow you to interact with Google Cloud Storage using the credentials associated with your account.
## Also import your Google Drive to access your team's Shared Google Drive as well as Operating System
from google.colab import auth
from google.colab import drive
import os
auth.authenticate_user()
drive.mount('/content/drive', force_remount=True)

## If your team is using a Shared Drive in a Google Workspace (usually through a University Gmail account)
directory_path = '/content/drive/Shared drives/Harvard Datathon 2024/'

In [ ]:
%%capture
# Prompt 1 | Question 1
import pandas as pd
from google.cloud import storage
import io

# COUNTY AVG AGE

client = storage.Client(project='HDSI-Agri-Datathon-2024')

#Initialize Paths
bucket_name = 'hdsi-agri-prompt-data'
filepath = "HDSI_AGRI_Prompt_2/prompt2_demos_landtotals_county.csv"

# Access the specified bucket
bucket = client.bucket(bucket_name)
blob = bucket.blob(filepath)

# Land Totals by County
content = blob.download_as_bytes()

# Load Data
raw_df = pd.read_csv(io.BytesIO(content))

# Cleaning Data


In [ ]:
%%capture

raw_df = raw_df[raw_df['COUNTY_CODE'] != 999]


print(raw_df['COUNTY_CODE'].dtype)
main_df = raw_df[['STATE_FIPS_CODE', 'COUNTY_CODE', 'YEAR', 'OPERATORS_PRINCIPAL_AGE_AVG_MEASURED_IN_YEARS', 'PRODUCERS_AGE_AVG_MEASURED_IN_YEARS']]
# Combine State FIP and County FIP
main_df['STATE_FIPS_CODE'] = raw_df['STATE_FIPS_CODE'].apply(lambda x: str(x).zfill(2))
main_df['COUNTY_CODE'] = raw_df['COUNTY_CODE'].apply(lambda x: str(x).zfill(3))
main_df['County FIP'] = main_df['STATE_FIPS_CODE'] + main_df['COUNTY_CODE']
main_df = main_df.sort_values(by='YEAR')
main_df['Avg_Age'] = main_df['OPERATORS_PRINCIPAL_AGE_AVG_MEASURED_IN_YEARS'].fillna(main_df['PRODUCERS_AGE_AVG_MEASURED_IN_YEARS'])
main_df = main_df.drop(['OPERATORS_PRINCIPAL_AGE_AVG_MEASURED_IN_YEARS', 'PRODUCERS_AGE_AVG_MEASURED_IN_YEARS', 'STATE_FIPS_CODE', 'COUNTY_CODE'], axis=1)
#
min_year = main_df['YEAR'].min()
max_year = main_df['YEAR'].max()

# Plotting Data

In [ ]:
import plotly.express as px
main_df = main_df[main_df['Avg_Age'] != '(D)']
main_df['Avg_Age'] = main_df['Avg_Age'].astype('float32')

# Calculate the min and max values of Avg_Age
min_age = main_df['Avg_Age'].min()
max_age = main_df['Avg_Age'].max()

fig = px.choropleth(main_df,
                    geojson='https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json',  # County boundaries
                    locations='County FIP',  # Column with FIPS codes
                    color='Avg_Age',  # Column with Avg Age data
                    scope='usa',  # Focus on USA
                    color_continuous_scale="Viridis",  # Color scale for the choropleth
                    animation_frame='YEAR',  # Use the 'Year' column to animate by year
                    range_color=(min_age, max_age),
                    title="Interactive Choropleth Map of Avg Age by County"
                   )
pd.options.mode.chained_assignment = None
# Update layout for better map visualization
fig.update_geos(fitbounds="locations", visible=False)

# Show the map
fig.show()

<ipython-input-5-cb54376b9d5a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['Avg_Age'] = main_df['Avg_Age'].astype('float32')
